In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.linalg import pinv
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
import io

# -------------------------
# A1: Units
# -------------------------
def summation_unit(x, w, b): return np.dot(x, w) + b
def step(x): return (np.array(x) >= 0).astype(float)
def bipolar_step(x): return np.where(np.array(x) >= 0, 1.0, -1.0)
def sigmoid(x): return 1 / (1 + np.exp(-np.array(x)))
def tanh(x): return np.tanh(x)
def relu(x): return np.maximum(0, np.array(x))
def leaky_relu(x, alpha=0.01): return np.where(np.array(x) > 0, x, alpha*np.array(x))
def comparator(y_true, y_pred): return y_true - y_pred

# -------------------------
# A2: Perceptron Trainer
# -------------------------
def perceptron_train(X, y, activation_fn, lr=0.05, w_init=[10,0.2,-0.75],
                     tol=0.002, max_epochs=1000):
    X, y = np.array(X), np.array(y)
    b, w = w_init[0], np.array(w_init[1:])
    errors=[]
    for epoch in range(max_epochs):
        sse=0
        for i in range(len(X)):
            z = summation_unit(X[i],w,b)
            y_pred = float(np.ravel(activation_fn(z))[0])
            err = y[i]-y_pred
            w += lr*err*X[i]
            b += lr*err
            sse += err**2
        errors.append(sse)
        if sse<=tol: break
    return w,b,errors,epoch+1

def plot_errors(errors,title):
    plt.figure()
    plt.plot(errors); plt.title(title)
    plt.xlabel("Epochs"); plt.ylabel("SSE"); plt.grid(True);
    plt.savefig(f'{title.replace(" ", "_")}.png')
    plt.close()


# -------------------------
# A3: Compare activations
# -------------------------
def compare_activations(X,y,activations):
    results={}
    for name,fn in activations:
        w,b,e,ep=perceptron_train(X,y,fn)
        results[name]=(w,b,e,ep)
    return results

# -------------------------
# A4: Vary learning rate
# -------------------------
def vary_lr(X,y,activation_fn,lrs):
    out={}
    for lr in lrs:
        _,_,_,ep=perceptron_train(X,y,activation_fn,lr=lr)
        out[lr]=ep
    return out

# -------------------------
# A6: Customer dataset (from lab sheet)
# -------------------------
def get_customer_data():
    data = {
        "Candies":[20,16,27,19,24,22,15,18,21,16],
        "Mangoes":[6,3,6,1,4,1,4,4,1,2],
        "Milk":[2,6,2,2,2,5,2,2,4,4],
        "Payment":[386,289,393,110,280,167,271,274,148,198],
        "High?":["Yes","Yes","Yes","No","Yes","No","Yes","Yes","No","No"]
    }
    df=pd.DataFrame(data)
    y=(df["High?"]=="Yes").astype(int).values
    X=df.drop(columns=["High?"]).values
    X=StandardScaler().fit_transform(X)
    return X,y

def sigmoid_train(X,y,lr=0.05,tol=0.002,max_epochs=10000):
    w=np.zeros(X.shape[1]); b=0; errors=[]
    for epoch in range(max_epochs):
        net=X.dot(w)+b; yhat=sigmoid(net)
        err=y-yhat; sse=np.sum(err**2); errors.append(sse)
        if sse<=tol: break
        grad=X.T.dot(yhat-y)/len(X); gradb=np.mean(yhat-y)
        w-=lr*grad; b-=lr*gradb
    return w,b,errors,epoch+1

# -------------------------
# A7: Pseudo-inverse
# -------------------------
def pseudo_inverse(X,y):
    X_aug=np.hstack([np.ones((X.shape[0],1)),X])
    return pinv(X_aug).dot(y)

# -------------------------
# A8–A10: Backprop
# -------------------------
def init_params(sizes):
    rng=np.random.default_rng(42)
    params=[]
    for i in range(len(sizes)-1):
        W=rng.normal(scale=0.5,size=(sizes[i+1],sizes[i]))
        b=np.zeros((sizes[i+1],))
        params.append([W,b])
    return params

def forward(X,params):
    a=X; acts=[a]; zs=[]
    for W,b in params:
        z=a.dot(W.T)+b; zs.append(z)
        a=sigmoid(z); acts.append(a)
    return zs,acts

def backprop(X,y,params,lr=0.05):
    n=len(X); zs,acts=forward(X,params)
    delta=(acts[-1]-y)*acts[-1]*(1-acts[-1]); deltas=[delta]
    for l in range(len(params)-1,0,-1):
        Wn,_=params[l]
        delta=(deltas[0].dot(Wn))*acts[l]*(1-acts[l])
        deltas.insert(0,delta)
    for i,(W,b) in enumerate(params):
        gradW=(deltas[i].T.dot(acts[i]))/n; gradb=np.mean(deltas[i],axis=0)
        params[i][0]-=lr*gradW; params[i][1]-=lr*gradb
    return np.sum((y-acts[-1])**2)

def train_backprop(X,y,sizes,lr=0.05,tol=0.002,max_epochs=1000):
    params=init_params(sizes); errors=[]
    for ep in range(max_epochs):
        sse=backprop(X,y,params,lr); errors.append(sse)
        if sse<=tol: break
    return params,errors,ep+1

# -------------------------
# A11 & A12: sklearn MLP
# -------------------------
def sklearn_mlp(X,y,hidden=(4,),activation='logistic'):
    clf=MLPClassifier(hidden_layer_sizes=hidden,activation=activation,
                        learning_rate_init=0.05,max_iter=2000,random_state=42)
    clf.fit(X,y); return clf

# -------------------------
# MAIN
# -------------------------
def main():
    X_and=np.array([[0,0],[0,1],[1,0],[1,1]]); y_and=np.array([0,0,0,1])
    X_xor=X_and; y_xor=np.array([0,1,1,0])

    # A2
    w,b,errs,ep=perceptron_train(X_and,y_and,step)
    print(f"A2 AND Step epochs: {ep}"); plot_errors(errs,"A2 AND Step")

    # A3
    acts=[("Step",step),("Bipolar",bipolar_step),
          ("Sigmoid",lambda x:(sigmoid(x)>=0.5).astype(float)),
          ("ReLU",lambda x:(relu(x)>=0.5).astype(float))]
    res=compare_activations(X_and,y_and,acts)
    for k,(w,b,e,ep) in res.items(): print(f"A3 {k} epochs: {ep}")

    # A4
    lr_epochs=vary_lr(X_and,y_and,step,[0.1*i for i in range(1,11)])
    print(f"A4 lr vs epochs: {lr_epochs}")

    # A5
    w,b,errs,ep=perceptron_train(X_xor,y_xor,step)
    print(f"A5 XOR Step epochs: {ep}")

    # A6
    Xc,yc=get_customer_data()
    w,b,errs,ep=sigmoid_train(Xc,yc)
    preds=(sigmoid(Xc.dot(w)+b)>=0.5).astype(int)
    print(f"A6 accuracy: {np.mean(preds==yc)}")

    # A7
    W=pseudo_inverse(Xc,yc); print(f"A7 pseudo-inverse weights: {W}")

    # A8 AND backprop
    _,errs,ep=train_backprop(X_and,y_and.reshape(-1,1),(2,2,1))
    print(f"A8 AND backprop epochs: {ep}")

    # A9 XOR backprop
    _,errs,ep=train_backprop(X_xor,y_xor.reshape(-1,1),(2,4,1))
    print(f"A9 XOR backprop epochs: {ep}")

    # A10 two-output AND
    y2=np.array([[1,0] if v==0 else [0,1] for v in y_and])
    _,errs,ep=train_backprop(X_and,y2,(2,3,2))
    print(f"A10 two-output epochs: {ep}")

    # A11 sklearn MLP on gates
    print(f"A11 AND preds: {sklearn_mlp(X_and,y_and).predict(X_and)}")
    print(f"A11 XOR preds: {sklearn_mlp(X_xor,y_xor).predict(X_xor)}")

    # A12: sklearn MLP on merged USGS dataset
    def load_usgs_data(filepath, value_name):
        """Loads and preprocesses a single USGS data file."""
        if not os.path.exists(filepath):
            print(f"Error: File '{filepath}' not found.")
            return pd.DataFrame()

        # Pre-process the file to remove surrounding quotes from each line
        cleaned_lines = []
        with open(filepath, 'r') as f:
            for line in f:
                # Strip whitespace and the quote characters
                cleaned_line = line.strip().strip('"')
                if cleaned_line and not cleaned_line.startswith('#'):
                    cleaned_lines.append(cleaned_line)

        if not cleaned_lines:
            print(f"Warning: File '{filepath}' is empty or only contains comments after cleaning.")
            return pd.DataFrame()

        # Join cleaned lines into a single string and wrap in a StringIO object
        cleaned_data_string = "\n".join(cleaned_lines)
        data_io = io.StringIO(cleaned_data_string)

        column_names = ['agency', 'site_no', 'datetime', 'tz', 'value', 'code']
        # Now read the cleaned, in-memory data
        df = pd.read_csv(data_io, sep='\t', header=None, names=column_names)

        # Now the standard filtering should work perfectly
        df = df[df['agency'] == 'USGS'].copy()

        if df.empty:
            print(f"Warning: No data with agency 'USGS' found in {filepath} after cleaning.")
            return pd.DataFrame()

        df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
        df[value_name] = pd.to_numeric(df['value'], errors='coerce')

        # Handle duplicate timestamps by averaging them
        df = df.groupby('datetime').mean(numeric_only=True)

        return df[[value_name]]

    # Load and merge the four datasets
    df_gage = load_usgs_data('gage_height.csv', 'gage_height')
    df_precip = load_usgs_data('precipitation.csv', 'precipitation')
    df_res = load_usgs_data('reservoir_storage.csv', 'reservoir_storage')
    df_stream = load_usgs_data('stream_flow.csv', 'stream_flow')

    df = pd.concat([df_gage, df_precip, df_res, df_stream], axis=1)

    if df.empty:
        print("Stopping execution: The merged dataframe is empty after loading the files.")
        return

    # Define target and features
    target_col = 'stream_flow'
    df['target'] = (df[target_col] > df[target_col].median()).astype(int)
    df.dropna(subset=['target'], inplace=True)

    X = df.drop(columns=[target_col, 'target'])
    y = df['target'].values
    X = X.fillna(X.mean()).values

    if X.shape[0] == 0:
        print("Stopping execution: Feature set is empty after processing.")
        return

    # Scale, split, train, and evaluate
    X = StandardScaler().fit_transform(X)
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)

    clf = sklearn_mlp(Xtr, ytr, hidden=(8,), activation='relu')
    print(f"A12 train acc: {clf.score(Xtr, ytr)} test acc: {clf.score(Xte, yte)}")


if __name__=="__main__":
    main()

A2 AND Step epochs: 130
A3 Step epochs: 130
A3 Bipolar epochs: 1000
A3 Sigmoid epochs: 130
A3 ReLU epochs: 124
A4 lr vs epochs: {0.1: 68, 0.2: 37, 0.30000000000000004: 23, 0.4: 23, 0.5: 19, 0.6000000000000001: 19, 0.7000000000000001: 15, 0.8: 14, 0.9: 13, 1.0: 12}
A5 XOR Step epochs: 1000
A6 accuracy: 1.0
A7 pseudo-inverse weights: [ 0.6        -0.09436819  0.21713405 -0.01342766  0.23416864]
A8 AND backprop epochs: 1000
A9 XOR backprop epochs: 1000
A10 two-output epochs: 1000
A11 AND preds: [0 0 0 1]
A11 XOR preds: [0 1 1 0]
A12 train acc: 0.9425255595468361 test acc: 0.945303867403315
